# Create and run Earthquake Estimator

In [31]:
!nvidia-smi

Sun Jan 27 00:42:26 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:81:00.0 Off |                    0 |
| N/A   38C    P0    32W / 250W |  15519MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import tensorflow as tf
import lstm_estimator
import earthquake_input_fn
from importlib import reload
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [3]:
def reload_tf():
    global lstm_estimator
    global earthquake_input_fn
    for i in range(2):
        import lstm_estimator
        import earthquake_input_fn
        reload(lstm_estimator)
        reload(earthquake_input_fn)
        del lstm_estimator
        del earthquake_input_fn
    import lstm_estimator
    import earthquake_input_fn
reload_tf()

In [7]:
def create_estimator_from_params(batch_size, timesteps, n_feats, feature_columns,
                                 lstm_cell_size, learning_rate, dropout_rate=0.5, lambda_l2_reg=0.0005,
                                 label_input_column=None, model_dir=None, eval_every=600):
    
    params = {
        'batch_size': batch_size,
        'timesteps': timesteps,
        'n_feats': n_feats,
        'feature_columns': feature_columns,
        'lstm_cell_size': lstm_cell_size,
        'learning_rate': learning_rate,
        'dropout_rate': dropout_rate,
        'lambda_l2_reg': lambda_l2_reg,
        'label_input_column': label_input_column
    }
    if model_dir is None:
        model_dir = '/workspace/persistent-data/models/%s' % (time.strftime('%Y-%m-%d-%H-%M-%S'))
        
    config = tf.estimator.RunConfig(model_dir=model_dir,
                                    log_step_count_steps=int(4096 / timesteps),
                                    save_checkpoints_secs=600)
    
    estim = tf.estimator.Estimator(model_fn=lstm_estimator.lstm_model_fn,
                                   params=params,
                                   model_dir=model_dir,
                                   config=config)
    return estim

In [8]:
def train_and_evaluate_estimator(earthquake_data_dir, scales, eval_every, eval_count, batch_size,
                                 timesteps, n_feats, feature_columns, lstm_cell_size, learning_rate,
                                 dropout_rate, lambda_l2_reg, model_dir=None):
    
    estim = create_estimator_from_params(batch_size, timesteps, n_feats, feature_columns=feature_columns,
                                         lstm_cell_size=lstm_cell_size, learning_rate=learning_rate,
                                         dropout_rate=dropout_rate, lambda_l2_reg=lambda_l2_reg,
                                         label_input_column=None, model_dir=model_dir, eval_every=eval_every)
    
    trainspec = tf.estimator.TrainSpec(input_fn=lambda: earthquake_input_fn.earthquake_input_fn2(earthquake_data_dir,
                                                                             batch_size,
                                                                             timesteps,
                                                                             scales=scales,
                                                                             traintest='train'),
                                       max_steps=1000000)
    
    evalspec = tf.estimator.EvalSpec(input_fn=lambda: earthquake_input_fn.earthquake_input_fn2(earthquake_data_dir,
                                                                           batch_size,
                                                                           timesteps,
                                                                           scales=scales,
                                                                           traintest='test'),
                                     steps=int(eval_count * (4096/timesteps)),
                                     start_delay_secs=1, throttle_secs=eval_every)
    
    return estim, trainspec, evalspec

In [40]:
reload_tf()

EARTHQUAKE_DATA_DIR = '/workspace/persistent-data/earthquake/tfrecords2'
BATCH_SIZE=64
TIMESTEPS=32
LSTM_CELL_SIZE=[64, 64, 64]
LEARNING_RATE=0.0005
DROPOUT_RATE=0.5
LAMBDA_L2_REG=0.0005
DISTINCT_BATCHES = 2030
STEPS_PER_BATCH = int(4096/TIMESTEPS)
EVAL_EVERY_N_SECONDS = 1800
EVAL_NUM_BATCHES = 100
N_FEATS = 1
SCALES = ['1e-1', '1e0', '1e1']
FEATURE_COLUMNS = [tf.feature_column.numeric_column(key='acousticdata', dtype=tf.int64, shape=(64,))]

estim, train_spec, eval_spec = train_and_evaluate_estimator(EARTHQUAKE_DATA_DIR, SCALES, EVAL_EVERY_N_SECONDS,
                                                            EVAL_NUM_BATCHES, BATCH_SIZE, TIMESTEPS, N_FEATS, FEATURE_COLUMNS,
                                                            LSTM_CELL_SIZE, LEARNING_RATE, DROPOUT_RATE, LAMBDA_L2_REG)

INFO:tensorflow:Using config: {'_experimental_distribute': None, '_num_worker_replicas': 1, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_train_distribute': None, '_global_id_in_cluster': 0, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_keep_checkpoint_max': 5, '_device_fn': None, '_protocol': None, '_task_id': 0, '_service': None, '_tf_random_seed': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_log_step_count_steps': 128, '_eval_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdb733ebf98>, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_is_chief': True, '_master': '', '_model_dir': '/workspace/persistent-data/models/2019-01-27-00-49-46'}


In [41]:
tf.estimator.train_and_evaluate(estim, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
FEATURES
{'acousticdata': <tf.Tensor 'IteratorGetNext:0' shape=(?, ?) dtype=int64>}
--------------------
LABELS
Tensor("IteratorGetNext:1", shape=(?, ?), dtype=float64, device=/device:CPU:0)
--------------------
MODE
train
--------------------
PARAMS
{'lstm_cell_size': [64, 64, 64], 'feature_columns': [_NumericColumn(key='acousticdata', shape=(64,), default_value=None, dtype=tf.int64, normalizer_fn=None)], 'learning_rate': 0.0005, 'lambda_l2_reg': 0.0005, 'batch_size': 64, 'dropout_rate': 0.5, 'label_input_column': None, 'timesteps': 32, 'n_feats': 1}
--------------------
INFO:tensorflow:Done calling model_fn.
INFO

KeyboardInterrupt: 